# `1` imports

In [23]:
import os
import dash
from dash import callback_context, ctx
# import dash_core_components as dcc
# import dash_html_components as html
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
import dash_daq as daq
from dash_bootstrap_templates import ThemeSwitchAIO

from dash.dependencies import Input, Output, State

import jupyter_dash

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.data as pld
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline


import warnings
import regex as re
warnings.filterwarnings('ignore')#to filter all the warnings
pd.set_option('float_format', '{:.4f}'.format)# to keep the float values short

# Import for wordcloud
from os import path

from PIL import Image

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import stylecloud

# `2` Clean data loading

In [24]:
df = pd.read_csv('Clean_Dataset_final.csv.zip')
df.reset_index(inplace = True)
df.drop_duplicates(subset = ['trending_date', 'video_id', 'country'], inplace = True)
df.dropna(subset = 'video_title', inplace = True)
df['trending_date'] =  pd.to_datetime(df['trending_date'])

df['video_title'] = df['video_title'].apply(lambda x : x[:15] + '...')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390412 entries, 0 to 401911
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   index           390412 non-null  int64         
 1   video_id        390412 non-null  object        
 2   published_at    390412 non-null  object        
 3   category_id     390412 non-null  int64         
 4   trending_date   390412 non-null  datetime64[ns]
 5   view_count      390412 non-null  int64         
 6   likes           390412 non-null  int64         
 7   dislikes        390412 non-null  int64         
 8   comment_count   390412 non-null  int64         
 9   thumbnail_link  390412 non-null  object        
 10  category_name   390412 non-null  object        
 11  country         390412 non-null  object        
 12  description     379076 non-null  object        
 13  tags            322489 non-null  object        
 14  video_title     390412 non-null  obj

# `3` plots

## `3a` First plot

In [26]:
def create_category_statistics(
    df,
    countries :list = ['Canada', 'USA', 'Great Britain'],
    metric = 'likes',
    dark = False
):
    df = df[df.country.isin(countries)].copy()
    
    if metric == 'video_id':
        category_comments = df.groupby(by = ['country','category_name'])[metric].count().reset_index()
        metric_label = 'videos'
    elif metric == 'comment_count':
        category_comments = df.groupby(by = ['country','category_name'])[metric].mean().reset_index()
        metric_label = 'comments'
    elif metric == 'view_count':
        category_comments = df.groupby(by = ['country','category_name'])[metric].mean().reset_index()
        metric_label = 'views'
    else:
        category_comments = df.groupby(by = ['country','category_name'])[metric].mean().reset_index()
        metric_label = metric


    category_comments['rank'] = category_comments.groupby('country')[metric].rank(ascending = False)
    category_comments = category_comments.sort_values(['country', 'rank'])

    lables = {'category_name' : 'video category', metric : 'number of ' + metric_label}
    fig1 = px.bar(
        data_frame = category_comments,
        x = 'category_name',
        y = metric,
        color ='country',
        barmode = 'group',
        labels = lables,
        hover_data = ['rank'],
        )
    
    fig1.update_layout(
        font = {'size' : 16}
    )
    
    if dark:
        fig1.update_layout(
            {
                'plot_bgcolor':'black',
                'paper_bgcolor':'black', 
                'font': {'color':'white'}
            }
        )

    return fig1
fig1= create_category_statistics(df, ['USA', 'Great Britain'], 'video_id')
fig1.show()

## `3b` Second plot

In [27]:
def create_trending_count_histogram(
    df,
    categories : list = ['Entertainment', 'Sports'],
    countries :list = ['Canada', 'USA', 'Great Britain'],
    dark = False
):

    df = df[df.country.isin(countries)].copy()
    df['appearance_count'] = df.groupby(by = 'video_id').cumcount() + 1
    
    appearance_df  = df.drop_duplicates('video_id', keep = 'last')
    appearance_df = appearance_df[appearance_df.category_name.isin(categories)]
    appearance_df = appearance_df.groupby(['category_name']).appearance_count.value_counts().reset_index(level = 0)

    appearance_df = appearance_df.sort_index()
    
    for category in categories:
        appearance_df.loc[appearance_df.category_name == category, 'appearance_count'] /= sum(appearance_df.loc[appearance_df.category_name == category, 'appearance_count'])
    
    labels = {'index' : 'no. of trending days', 'appearance_count': '% of videos'}

    fig2 = px.area(
        data_frame = appearance_df,
        x = appearance_df.index,
        y = 'appearance_count',
        color = 'category_name',
        range_x=[1, 40],
        labels = labels,
    )
    
    fig2.update_layout(
        font = {'size' : 16}
    )
    
    if dark:
        fig2.update_layout(
            {
                'plot_bgcolor':'black',
                'paper_bgcolor':'black',
                'font': {'color':'white'}
            }
        )
        
    return fig2
    
create_trending_count_histogram(df, ['Sports'])

## `3c` Third plot and its child

In [28]:
def create_timeseries_trend(
    df,
    countries : list = ['USA', 'Canada', 'Great Britain'],
    dark = False
):
    df_copy = df[df.country.isin(countries)].copy()
    
    df_copy = df_copy.drop_duplicates(subset=['country', 'video_id'], keep='last')
    df_copy_category_counts = df_copy.groupby(['country', 'trending_date','category_name'], as_index=False)['view_count'].sum()

    df_copy_category_counts['trending_date'] = pd.to_datetime(df_copy_category_counts['trending_date']).dt.date

    df_copy_news_count = df_copy_category_counts[df_copy_category_counts.category_name.isin(['News & Politics'])]
    
    labels = {'view_count': 'View Count (Millions)', 'trending_date': 'Trending Date'}
    
    fig3 = px.line(
        df_copy_news_count,
        x='trending_date',
        y='view_count',
        title='View Count Time Series for category: News & Politics (USA)',
        labels=labels,
        color = 'country',
        hover_data= ['country', 'trending_date', 'category_name']
    )

    
    fig3.update_layout(
        font = {'size' : 16}
    )
    
    fig3.update_xaxes(rangeslider_visible=True)
    fig3.update_layout(title_x=0.5)
    
    if dark:
        fig3.update_layout(
            {
                'plot_bgcolor':'black',
                'paper_bgcolor':'black',
                'font': {'color':'white'}
            })
    
    return fig3

create_timeseries_trend(df)

In [29]:
from datetime import date, timedelta

plt.rcParams['figure.figsize'] = [10, 5]

# Adding unwanted words and social media tags to stopword list:
stopwords = set(STOPWORDS)
stopwords.update(['follow', 'twitter', 'social', 'instagram', 'subscribe', 'snapchat', 'youtube', 'videos', 'video'\
                  ,'channel', 'share', 'facebook', 'comment', 'like', 'take', 'go', 'got', 'back',\
                  'much', 'made', 'keep', 'watch','none', 'check', 'will', 'make'])


def generate_wordcloud_text(df, country : str, category : str, date_start, date_end):
    
    df = df[df.country == country]
    
    dateMask = (df.trending_date > date_start) & (df.trending_date < date_end) & (df.country == country)
    
    tag_text = " ".join(str(text) for text in df.tags[(df.category_name == category ) & (dateMask)])
    title_text = " ".join(str(text) for text in df.video_title[(df.category_name == category ) & (dateMask) ])
    tag_title_text = tag_text + ' ' + title_text
    
    return tag_title_text


def generate_wordcloud_form_text(
    df,
    country : str,
    category : str,
    date_start,
    date_end,
    stop_words,
    img_name,
    dark = False,
):
    
    text = generate_wordcloud_text(df, country, category, date_start, date_end)
    
    stylecloud.gen_stylecloud(
        text,
        icon_name='fab fa-youtube',
        #colors='white',
        #background_color='black',
        output_name = img_name,
        #collocations=False,
        stopwords = stop_words
                         )
    img = Image.open(img_name)
    fig = px.imshow(
        img,
        title = f'trending in {country} between {str(date_start)[:11]} and {str(date_end)[:11]}',
        width = 1200,
        height = 900,
                   )
    
    fig.update_layout(coloraxis_showscale=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    
    if dark:
        fig.update_layout(
            {
                'plot_bgcolor':'black',
                'paper_bgcolor':'black', 
                'font': {'color':'white'}
            })

    return fig

## fourth plot : video trending progression

In [30]:
df.sample(1).iloc[0, :]['video_title']

'how i beat brea...'

In [31]:
def create_video_view_progression(
    df,
    video_title,
    dark = False
):
    df = df[df.video_title == video_title].drop_duplicates('trending_date')
    df = df.sort_values('trending_date')
    
    fig = px.line(
        data_frame = df,
        x = 'trending_date',
        y = 'view_count',
        markers=True
    )

    
    fig.update_layout(
        font = {'size' : 16}
    )
    
    if dark:
        fig.update_layout(
            {
                'plot_bgcolor':'black',
                'paper_bgcolor':'black', 
                'font': {'color':'white'}
            })        
    return fig

In [32]:
create_video_view_progression(df, 'ant  decs under...')

## fifth plot

In [33]:
race_df = pd.read_csv('USA_race.csv')
race_df = race_df[race_df.date > race_df.date[30]]
race_df.channel_title = race_df.channel_title.apply(lambda x : x[:15] + '...')

In [34]:
def create_channel_race(
    race_df,
    dark = False
):
    
    fig = px.bar(
        data_frame = race_df,
        x = 'video_count',
        y = 'channel_title',
        animation_frame = 'date',
        color = 'video_count'
    )
    
    if dark:
        fig.update_layout(
            {
                'plot_bgcolor':'black',
                'paper_bgcolor':'black', 
                'font': {'color':'white'}
            })       
        
    return fig



In [35]:
create_channel_race(race_df, dark = False)

# `4` dashboard design

## Logo

In [36]:
logo = html.Img(
    
#     src = app.get_asset_url('title2.png'),
    src = r'https://logos-world.net/wp-content/uploads/2020/04/YouTube-Logo.png',
#     style={'margin':'auto'},
    width = 384,
    height = 216,
)

In [37]:
# logo = html.Img(
    
#     src = 'https://all-4-one.com/wp-content/uploads/2020/12/youtube-icon-logo-logo-icon-png-svg.png', 
#     style={'margin':'40px 40px 20px 20px',
#           'display':'inline-block'},
#     width = 150,
#     height = 90
# )

## BANs

In [38]:
sum_views = df.sort_values('view_count', ascending = False).drop_duplicates('video_id')['view_count'].sum()
BANs = dbc.Row(
        [
            dbc.Col(
                [
                    dbc.Card(
                        [
                            html.H2(
                                f'{len(df.trending_date.unique())} days',
                                className = 'text-center',
                                id = 'ban_1'
                            )
                        ],
                        color = 'danger',
                        outline=True,
                    )      
                ],
                width = {'offset' : 2, "size" : 2}
            ),
            
            dbc.Col(
                [
                    dbc.Card(
                        [
                            html.H2(
                                f'{len(df.video_id.unique())//1000}K videos',
                                className = 'text-center',
                                id = 'ban_2'

                            )
                        ],
                        color = 'danger',
                        outline=True
                    )      
                ],
                width = 3
            ),
            dbc.Col(
                [
                    dbc.Card(
                        [
                            html.H2(
                                f'{sum_views//1_000_000_000}B views',
                                className = 'text-center',
                                id = 'ban_3'

                            )
                        ],
                        color = 'danger',
                        outline=True
                    )      
                ],
                width = 3

            ),
        ],
    )

## Buttons

In [39]:
Buttons = html.Div(
        [
            dbc.Button(
                html.H1("All Countries"),
                id = 'ALL',
                className="me-md-2 bg-light text-dark",
                style = {'width' : '300px', 'height' : '150px', 'fonsize' : '20'}
            ),
            dbc.Button(
                html.H1("USA"),
                id = 'USA',
                className="me-md-2 bg-light text-dark",
                style = {'width' : '300px', 'height' : '150px', 'fonsize' : '20'}
            ),
            dbc.Button(
                html.H1("Canada "),
                id = 'CA',
                className="me-md-2 bg-light text-dark",
                style = {'width' : '300px', 'height' : '150px', 'fonsize' : '20'}
            ),
            dbc.Button(
                html.H1("Great Britain "),
                id = 'GB',
                className="me-md-2 bg-light text-dark",
                style = {'width' : '300px', 'height' : '150px', 'fonsize' : '20'}
            ),
        ],
        
        className="d-grid gap-2 d-md-flex justify-content-md-center",
    )

## video search

In [40]:
search = \
dcc.Dropdown(
    options= [{'label': x, 'value': x} for x in df.video_title.drop_duplicates()],
    id='search_desc',
    # Ensure input is triggered with 'Enter'
    # Ensure the plot can load without a selection
#     required = False,
    style={'width':'600px', 'height':'90px', 'margin' : 'auto', 'align' : 'center'}

)

## Graphs

In [42]:
Graph_layout = \
[
    dbc.Row(
        [
            dbc.Col(
                [
                    #first rwo first graph
                    dbc.Card(
                        [
                            html.H3(
                                'most popular video categories by',
                                style = {'display': 'inline-block', 'margin' : 'auto'},
                            
                            ),
                            dcc.Dropdown(
                                [
                                    'videos',
                                    'views',
                                    'likes',
                                    'comments'
                                ],
                                id = 'category_features'

                            ),
                            dcc.Graph(              
                                id = 'popular_categories',
                                style = {'height' : '800px'}
                            )

                        ],
                        className = "shadow-lg"
                        
                    )      
                ],
                width = 6
            ),
            
            dbc.Col(
                [
                    #first rwo second Graph
                    dbc.Card(
                        [     
                            html.H3(
                                'channels with most trending videos over time',
                                style = {'display': 'inline-block', 'margin' : 'auto'},

                            ),
                            dcc.Graph(              
                                id = 'channel_race',
                                style = {'height' : '900px'}
                            )

                        ],
                        className = "shadow-lg"

                    )      
                ],
                width = 6,

            ),
        ],
        
        
    ),
    
    html.Br(),
    
    dbc.Row(
        [
            #second rwo first graph
            dbc.Col(
                [
                    dbc.Card(
                        [ 
                            dcc.Graph(              
                                id = 'timeseries',
                                style = {'height' : '800px'}
                            )

                        ],
                        className = "shadow-lg"
                    )
                    

                ],
                width = 8

            ),
            
            #second rwo second graph
            dbc.Col(
                [
                    dbc.Card(
                        
                          dcc.Graph(
                            id = 'wordcloud',
                            style = {'height' : '800px'}

                        )                   
                    )

                ],
                width = 4
            ),
        ],
    ),
    html.Br(),
    dbc.Row(
        [
            #third rwo first graph
            dbc.Col(
                [
                    dbc.Card(
                        [ 
                            html.Img(

                                src = 'https://wallsdesk.com/wp-content/uploads/2016/06/YouTube-Images.jpg', 
                                
                                width = 600,
                                height = 450,
                                style = {'margin' : 'auto', 'align' : 'center', 'display':'inline-block'},
                                id = 'thumbnail'

                            ),
                            html.Br(),
                            search
                        ],                        
                    )      
                ],
                style = {'margin' : 'auto', 'align' : 'center'},
                width = 4,

            ),
            
            #second rwo second graph
            dbc.Col(
                [
                    dbc.Card(
                        [
                            html.H3(
                                id = 'video_desc',
                                className = 'text-center'
                            ),
                            
                            html.Div(id = 'information_table'),
                            dcc.Graph(id = 'trend_progression')
                        ]
                    )
                ],
                width = 8
            ),
        ],
    ),
    
]


## app instantiation

In [43]:
app = jupyter_dash.JupyterDash(external_stylesheets = [dbc.themes.LITERA])

## overall layout

In [44]:
layout = \
[
    # Title row
    html.Br(),
    dbc.Row(
        
        [   
            dbc.Col(
                [
                    logo,
                    html.H2('YouTube', style = {'display': 'inline-block', 'margin':'20px 20px 5px 5px'}),
                    html.H4('Trendings',style={'color':'Tomato', 'display':'inline-block', 'margin':'20px 20px 5px 5px'})

                ], 
                style = {'display':'inline-block', 'margin' : 'auto'},
                width = {'offset' : 2},
            ),
            dbc.Col(
                [
                    daq.ToggleSwitch(
                        id="theme",
                        size = 70
                    )
                ], 
                align = 'right',
                width = 1
            ),
        ]
    
    ),
    
    # BANs row
    html.Hr(style={'borderWidth': "0.3vh", "width": "100%", "color": 'gray'}), 
    BANs,
    
    # Buttons
    html.Hr(style={'borderWidth': "0.3vh", "width": "100%", "color": 'gray'}), 
    Buttons,
    html.Hr(style={'borderWidth': "0.3vh", "width": "100%", "color": 'gray'}),
    
    #Grpahs
    *Graph_layout,
    
    html.Br()
]

app.layout = html.Div(children = layout, id = 'base')

## callbacks

In [45]:
#conserving latest figure states
class Latest:
    def __init__(self,
                 latest_countries = ['USA', 'Canada', 'Great Britain'],
                 latest_metric = 'video_id',
                 latest_theme = False,
                 last_hover = None,
                 last_search = None
                ):
        self.latest_countries = latest_countries
        self.lateset_metric = latest_metric
        self.latest_theme = latest_theme
        self.last_hover = last_hover
        self.last_search = last_search
        
state = Latest()

In [46]:
# for first figure
# country, theme and category feature adjustment

@app.callback(
    Output('popular_categories', 'figure'),
    
    Input(component_id = 'theme', component_property ='value'),
    Input(component_id = 'USA', component_property ='n_clicks'),
    Input(component_id = 'CA', component_property ='n_clicks'),
    Input(component_id = 'GB', component_property ='n_clicks'),
    Input(component_id = 'ALL', component_property ='n_clicks'),
    Input(component_id = 'category_features', component_property = 'value'),

#     prevent_initial_call=True
)
def category_stats(theme, us_clicks, ca_clicks, gb_clicks, all_clicks, slider_value):

    trigger = ctx.triggered_id

    if trigger == None:
        fig = create_category_statistics(df,countries = state.latest_countries, metric = state.lateset_metric, dark = state.latest_theme)
        return fig
    
    if trigger == 'theme':
        state.latest_theme = theme
        fig = create_category_statistics(df,countries = state.latest_countries, metric = state.lateset_metric, dark = state.latest_theme)
        return fig
    
    elif trigger != 'category_features':
        if trigger == 'USA':
            countries = ['USA']
        elif trigger == 'CA':
            countries = ['Canada']
        elif trigger == 'GB':
            countries = ['Great Britain']
        elif trigger =='ALL':
            countries = ['USA', 'Canada', 'Great Britain']
            
        state.latest_countries = countries
        fig = create_category_statistics(df,countries = countries, metric = state.lateset_metric, dark = state.latest_theme)
        return fig
    else:
        
        if slider_value == 'videos' : 
            fig =  create_category_statistics(df, countries = state.latest_countries, metric = 'video_id', dark = state.latest_theme) 
            state.lateset_metric = 'video_id'
        elif slider_value == 'views':
            fig =  create_category_statistics(df, countries = state.latest_countries, metric = 'view_count', dark = state.latest_theme)
            state.lateset_metric = 'view_count'
        elif slider_value == 'comments':
            fig =  create_category_statistics(df, countries = state.latest_countries, metric = 'comment_count', dark = state.latest_theme)
            state.lateset_metric = 'comment_count'
        elif slider_value == 'likes':
            fig =  create_category_statistics(df, countries = state.latest_countries, metric = 'likes', dark = state.latest_theme)
            state.lateset_metric = 'likes'

        return fig
    

In [47]:
# for second figure
# country and theme adjustment

@app.callback(
    Output('channel_race', 'figure'),
    
    Input(component_id = 'theme', component_property ='value'),
    Input(component_id = 'USA', component_property ='n_clicks'),
    Input(component_id = 'CA', component_property ='n_clicks'),
    Input(component_id = 'GB', component_property ='n_clicks'),
    Input(component_id = 'ALL', component_property ='n_clicks'),

#     prevent_initial_call=True
)
def trending_time(theme, us_clicks, ca_clicks, gb_clicks, all_clicks):

    trigger = ctx.triggered_id

    if trigger == None:
        fig = create_channel_race(race_df, dark = state.latest_theme)
        return fig
    
    if trigger == 'theme':
        state.latest_theme = theme
        fig = create_channel_race(race_df, dark = state.latest_theme)
        return fig
    
    elif trigger != 'category_features':
        if trigger == 'USA':
            countries = ['USA']
        elif trigger == 'CA':
            countries = ['Canada']
        elif trigger == 'GB':
            countries = ['Great Britain']
        elif trigger =='ALL':
            countries = ['USA', 'Canada', 'Great Britain']
            
        state.latest_countries = countries
        fig = create_channel_race(race_df, dark = state.latest_theme)
        return fig

In [48]:
# for third figure
# country and theme adjustment

@app.callback(
    Output('timeseries', 'figure'),
    
    Input(component_id = 'theme', component_property ='value'),
    Input(component_id = 'USA', component_property ='n_clicks'),
    Input(component_id = 'CA', component_property ='n_clicks'),
    Input(component_id = 'GB', component_property ='n_clicks'),
    Input(component_id = 'ALL', component_property ='n_clicks'),

#     prevent_initial_call=True
)
def timeseries_trends(theme, us_clicks, ca_clicks, gb_clicks, all_clicks):

    trigger = ctx.triggered_id
    
    if trigger == None:
        fig = create_timeseries_trend(df,countries = state.latest_countries, dark = state.latest_theme)
        return fig
    
    if trigger == 'theme':
        state.latest_theme = theme
        fig = create_timeseries_trend(df,countries = state.latest_countries, dark = state.latest_theme)
        return fig
    
    elif trigger != 'category_features':
        if trigger == 'USA':
            countries = ['USA']
        elif trigger == 'CA':
            countries = ['Canada']
        elif trigger == 'GB':
            countries = ['Great Britain']
        elif trigger =='ALL':
            countries = ['USA', 'Canada', 'Great Britain']
            
        state.latest_countries = countries
        fig = create_timeseries_trend(df,countries = countries, dark = state.latest_theme)
        return fig

In [49]:
# for entire dashboard
# theme adjustment

@app.callback(
    Output('base', 'style'),
    Output('ban_1', 'style'),
    Output('ban_2', 'style'),
    Output('ban_3', 'style'),
    
    Input(component_id = 'theme', component_property ='value'),
    
    prevent_initial_call=True
)
def timeseries_trends(theme):

    trigger = ctx.triggered_id

    state.latest_theme = theme
    if theme:
        style = {'background-color': 'black', 'color' : 'white'}
        
    else:
        style = {'background-color': 'white', 'color' : '#282828'}
    return style, style, style, style

In [50]:
#wordcloud generation

@app.callback(
    Output(component_id = 'wordcloud', component_property = 'figure'),
    
    State(component_id = 'timeseries', component_property = 'hoverData'),
    
    Input(component_id = 'timeseries', component_property = 'clickData'),
    Input(component_id = 'theme', component_property ='value'),

    prevent_initial_call = True
)
def timely_wordcloud(hover_data, click_data, theme):

    trigger = ctx.triggered_id
    
    if trigger == 'timeseries':
        if hover_data == None : 
            return dash.no_update
        else:
            state.last_hover = hover_data['points'][0]
            hover_data = hover_data['points'][0]
    else :
        hover_data = state.last_hover
        
    if hover_data == None:
        return dash.no_update
    
    year, month, day = hover_data['x'].split('-')
    start_date = pd.Timestamp(year = int(year), month = int(month), day= int(day)) - pd.Timedelta(4, unit = 'day')
    end_date = pd.Timestamp(year = int(year), month = int(month), day= int(day)) + pd.Timedelta(4, unit = 'day')
    
    cloud_fig = generate_wordcloud_form_text(
        df,
        hover_data['customdata'][0],
        hover_data['customdata'][1],
        start_date,
        end_date,
        stopwords,
        'cloud.png',
        dark = state.latest_theme
    )

    return cloud_fig

In [51]:
@app.callback(
    Output(component_id='information_table', component_property= 'children'),
    Output(component_id='video_desc', component_property= 'children'),
    Output(component_id='trend_progression', component_property= 'figure'),
    Output(component_id='thumbnail', component_property= 'src'),
    
    Input(component_id = 'theme', component_property ='value'),
    Input(component_id='search_desc', component_property='value'),
    
    prevent_initial_call=True

)
def update_plot(theme, search_value):

    trigger = ctx.triggered_id
    
    if trigger == 'theme':
        if state.last_search:
            return state.last_search[0], state.last_search[1], create_video_view_progression(df, state.last_search[1], dark = True), state.last_search[3]
        else:
            return dash.no_update, dash.no_update, dash.no_update, dash.no_update

    if search_value == '' or search_value == None:
        return dash.no_update, dash.no_update, dash.no_update, dash.no_update
    
    # Undertake the filter here using the user input
    if search_value:
        title = df[df.video_title.str.contains(search_value, case=False)]['video_title'].iloc[0]
        
        thumbnail = df[df.video_title.str.contains(search_value, case=False)]['thumbnail_link'].iloc[0]
        
        filtered = df[df['video_title'].str.contains(search_value, case=False)][['view_count', 'likes', 'dislikes', 'category_name']]
    
    if len(filtered) == 0:
        return dash.no_update, dash.no_update, dash.no_update, dash_no_update
    
    filtered = filtered.iloc[[0], :]
    filtered = pd.DataFrame([filtered.iloc[0, :].T], columns = filtered.columns)    

    table = dbc.Table.from_dataframe(filtered, striped=True, bordered=True, hover=True)
    
    fig = create_video_view_progression(df, title)
    
    state.last_search = (table, title, fig, thumbnail)
    
    return table, title, fig, thumbnail

## server run

In [52]:
app.run_server(
    port = 8055,
    debug = True
)

Dash app running on http://127.0.0.1:8055/
